In [1]:
#%pip install -r dinov2/requirements.txt

In [1]:
## train python script

#%pip install torchvision
#%pip install opencv-python

from pathlib import Path

from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor

from define_dataset import CloudDataset, target_transform


IMG_PATH = Path('/gws/nopw/j04/iecdt/computer-vision-data/cam_a')
PATH = Path('/gws/nopw/j04/iecdt/JERMIT_the_frog/')


def train():    
    dataset = CloudDataset(
        PATH / 'hydrometeors_time_aligned_classes.csv', 
        IMG_PATH / 'compressed_rectified_imgs', 
        transform=ToTensor(),
        target_transform=target_transform
    )

    training_dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

    for images, labels in training_dataloader:
        print(images.shape, labels.shape)
        break

In [2]:
#%pip install transformers
#%pip install pillow

from transformers import AutoImageProcessor, Dinov2ForImageClassification
import torch
import PIL 

# Load the pre-trained model and image processor
processor = AutoImageProcessor.from_pretrained("facebook/dinov2-base")
model = Dinov2ForImageClassification.from_pretrained("facebook/dinov2-base")

# Set the model to evaluation mode
model.eval()

/home/users/rsimpson/iecdt-computer-vision-group-proj/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Some weights of Dinov2ForImageClassification were not initialized from the model checkpoint at facebook/dinov2-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Dinov2ForImageClassification(
  (dinov2): Dinov2Model(
    (embeddings): Dinov2Embeddings(
      (patch_embeddings): Dinov2PatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(14, 14), stride=(14, 14))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): Dinov2Encoder(
      (layer): ModuleList(
        (0-11): 12 x Dinov2Layer(
          (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
          (attention): Dinov2SdpaAttention(
            (attention): Dinov2SdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): Dinov2SelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)

In [3]:
## modifying the model for my depth

import torch.nn as nn

# Modify the classifier to match your output size (459 × 2) if doing 1-hot encoding
# num_classes = 459 * 2
num_classes = 459
model.classifier = nn.Linear(model.classifier.in_features, num_classes)


In [4]:
import torch.optim as optim

# criterion = nn.CrossEntropyLoss()  # Use for classification
criterion = nn.BCEWithLogitsLoss()  # Use for classification with only two classes
optimizer = optim.Adam(model.parameters(), lr=1e-4)


In [5]:
# Move model to the appropriate device (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define number of epochs and batch size
num_epochs = 10  # You can adjust this based on how long you want to train
batch_size = 16

dataset = CloudDataset(
        PATH / 'hydrometeors_time_aligned_classes.csv', 
        IMG_PATH / 'compressed_rectified_imgs', 
        transform=ToTensor(),
        target_transform=target_transform
    )

training_dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

# Training loop
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    total_loss = 0

    for images, labels in training_dataloader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()  # Zero the gradients before the backward pass
        outputs = model(images).logits  # Pass images through the model
        print(outputs.shape, labels.shape)
        loss = criterion(outputs, labels)  # Compute the loss
        loss.backward()  # Backpropagate the loss
        optimizer.step()  # Update model parameters

        total_loss += loss.item()  # Accumulate loss for this batch

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss:.4f}")

    train()


torch.Size([16, 459]) torch.Size([16, 459])
torch.Size([16, 459]) torch.Size([16, 459])
torch.Size([16, 459]) torch.Size([16, 459])


KeyboardInterrupt: 

In [4]:
import pandas as pd
label_data_csv = pd.read_csv(f'{PATH}/hydrometeors_time_aligned_classes.csv')
label_data_csv

,height,1691388010,1691388070,1691388130,1691388190,1691388250,1691388310,1691388370,1691388430,1691388490,...,1693155020,1693155080,1693155140,1693155200,1693155260,1693155320,1693155380,1693155440,1693155500,1693155560
0,114.54125,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,144.51833,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,174.49533,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,204.47241,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,234.44948,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
454,13724.13100,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
455,13754.10800,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
456,13784.08500,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
457,13814.06200,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
dataset = CloudDataset(
        PATH / 'hydrometeors_time_aligned_classes.csv', 
        IMG_PATH / 'compressed_rectified_imgs', 
        transform=ToTensor(),
        target_transform=target_transform
    )

training_dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

In [7]:
data, label = next(iter(training_dataloader))

In [8]:
label.shape

torch.Size([16, 459, 2])

In [33]:
data.shape

torch.Size([16, 3, 504, 504])